In [1]:
import requests
from bs4 import BeautifulSoup
import time
import json
import csv
import random
from tqdm import tqdm

In [2]:
request_headers = {
        "method": "GET",
        "http_version": "HTTP/2",
        "Host": "xistore.by",
        "User-Agent": "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:134.0) Gecko/20100101 Firefox/134.0",
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
        "Accept-Language": "ru-RU,ru;q=0.8,en-US;q=0.5,en;q=0.3",
        "Accept-Encoding": "gzip, deflate, br, zstd",
        "DNT": "1",
        "Sec-GPC": "1",
        "Connection": "keep-alive",
        "Cookie": "hg-client-security=2raddTr8odLrsgYb67RiYQe1LA9; PHPSESSID=wJ2S6mmhESmmdK1Z148IspYnFyPx3lD2; VISIT_USER_ID=BA2F506F-4E40-C9BA-82FF-E3A1546CDCA6; BITRIX_SM_5_SALE_UID=123462632; xistore_banner_show=167986; _gcl_au=1.1.628187606.1736801209; _ga_63ZME06VVY=GS1.1.1736801209.1.0.1736801209.60.0.0; _ga=GA1.1.83084367.1736801210",
        "Upgrade-Insecure-Requests": "1",
        "Sec-Fetch-Dest": "document",
        "Sec-Fetch-Mode": "navigate",
        "Sec-Fetch-Site": "cross-site",
        "Referer": "https://xistore.by/catalog/telefony/?PAGEN_1=1",
        "Priority": "u=0, i"
}

In [8]:
all_phones_links = []

# XI-store parser
# Phone catalog pages parser
for i in range(1, 6):
    url = f"https://xistore.by/catalog/smart_televizory/?PAGEN_1={i}"
    r = requests.get(url, headers=request_headers)
    soup = BeautifulSoup(r.content, 'html.parser')
    # Find all phone links on the page
    for a in soup.find_all('a', class_='search__page_item-name'):
        href = a.attrs['href']
        absolute_url = 'https://xistore.by' + href
        all_phones_links.append(absolute_url)
        print(len(all_phones_links))
    time.sleep(random.randint(1, 3))

# /catalog/apple/smartfon_iphone_14_pro/
# https://xistore.by/catalog/apple/smartfon_iphone_14_pro/

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143


In [9]:
all_phones_links = list(set(all_phones_links))

In [10]:
def save_to_csv(product_data_list, filename='notebooks.csv'):
    with open(filename, 'w', newline='', encoding='utf-8') as csvfile:
        fieldnames = ['name', 'description', 'reviews', 'shops_availability', 'price', 'full_price', 'brand', 'remind_status', 'url', 'category', 'images', 'characteristics'] 
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

        writer.writeheader()
        for product_data in product_data_list:
            # Convert lists and dictionaries to JSON strings for CSV storage
            product_data['reviews'] = json.dumps(product_data['reviews'])
            product_data['shops_availability'] = json.dumps(product_data['shops_availability'])

            product_data['images'] = json.dumps(product_data['images'])
            product_data['characteristics'] = json.dumps(product_data['characteristics'])
            writer.writerow(product_data)

In [6]:
all_product_data = []

# Collect detail information
for link in tqdm(all_phones_links):
    print(link)
    try:
        response = requests.get(link, headers=request_headers)
        response.raise_for_status()  # Raise an exception for bad status codes
        soup = BeautifulSoup(response.content, 'html.parser')

        product_data = {}

        # Name
        product_data['name'] = soup.find('h1').text.strip()

        try:
            description = soup.find('div', class_='detail-text-description basic--content').text.strip()
        except:
            meta_description = soup.find('meta', attrs={'itemprop': 'description'})
            description = meta_description['content'].strip() if meta_description else ""
            
        product_data['description'] = description

        # Reviews (currently placeholder, needs more complex logic to scrape reviews)
        product_data['reviews'] = [] # Placeholder for now,  

        # Shops availability (requires more sophisticated logic if data isn't simply embedded)
        availability_scripts = soup.find_all('div', class_=lambda text: text and 'wrapped-city city-wrap' in text)
        shops_list = []
        if availability_scripts:
            for script in availability_scripts:
                shop_data = {}
                try: 
                    shop_data['city'] = script.find('div', class_='pa-heading').text.strip()
                    for shop_block in script.find_all('div', class_='pa-result_item'):
                        shop_data['name'] = shop_block.find('div', class_='pa-result_title').text.strip()
                        shop_availability = shop_block.find('div', class_=lambda x: x and x.startswith('count-code'))
                        if shop_availability.attrs['class'][0].endswith('available-res'):
                            shop_data['availability'] = False
                        else:
                            shop_data['availability'] = True
                        shop_data['worktime'] = shop_block.find('div', class_='pa-result_time').find('p', class_='pa-result_info-data').text.strip()
                        shop_data['adress'] = shop_block.find('div', class_='pa-result_adress').find('p', class_='pa-result_info-data').text.strip()
                        # print(shop_data)
                        shops_list.append(shop_data.copy())
                except (IndexError, json.JSONDecodeError):
                    product_data['shops_availability'] = {} # Handle errors gracefully
                    print(f"Error parsing availability data for {url}")
                
        product_data['shops_availability'] = shops_list

        # Price (needs better error handling)
        price_element = soup.find('span', class_='count price-color')
        if price_element:
            price_str = price_element.text.replace(' ', '').replace('<sup>', '.').replace('</sup>', '').strip()
            product_data['price'] = float(price_str) if price_str else None
        
        # Discount
        old_price_element = soup.find('span', class_='old-price')
        if old_price_element and price_element:
            old_price_str = old_price_element.text.replace(' ', '').replace('<sup>', '.').replace('</sup>', '').strip()
            old_price = float(old_price_str) if old_price_str else 0.0
            product_data['full_price'] = old_price
        else:
            product_data['full_price'] = 0.0

        # Brand (meta tag)
        product_data['brand'] = soup.find('meta', attrs={'itemprop': 'name'})['content']

        # Remind status (placeholder -  logic depends on how it's indicated on the page)
        product_data['remind_status'] = any([shop['availability'] for shop in product_data['shops_availability']])

        # URL
        product_data['url'] = url
        
        # Category
        category_element = soup.find('a', class_='current') # Assuming category in breadcrumb
        product_data['category'] = category_element.text.strip() if category_element else "Unknown"


        # Images
        product_data['images'] = [img['src'] for img in soup.find_all('img', itemprop='image')]


        # Characteristics 
        product_data['characteristics'] = []
        characteristics_containers = soup.select('.characteristic--list')  # Select all characteristic lists
        for container in characteristics_containers:
            for item in container.select('.characteristic--item'):  # Iterate through characteristics within each list
                name = item.select_one('.name').text.strip()
                value = item.select_one('.characteristic').text.strip()
                product_data['characteristics'].append({'name': name, 'value': value})
                
        print(product_data)
        time.sleep(random.randint(1, 3))
        all_product_data.append(product_data)
    except requests.exceptions.RequestException as e:
        print(f"Error fetching URL {link}: {e}")

  0%|          | 0/38 [00:00<?, ?it/s]

https://xistore.by/catalog/smart_televizory/televizor_xiaomi_a2_43_fhd/
{'name': 'Телевизор Xiaomi A2 43" FHD', 'description': 'Кинотеатр у\xa0вас дома\n\r\n    Хотите смотреть сериалы или другой контент по-новому? Это легко, если купить телевизор Xiaomi A2. Премиальные 4К\xa0дисплей 3840×2160 пикселей или FHD дисплей 1920×1080 пикселей переворачивают представления о\xa0качестве\r\n    изображения. Все цвета реалистичные, а\xa0динамичные кадры плавные. Технологии Dolby Vision и\xa0MEMC обеспечивают полное погружение.\r\n\n\n\n\n\n\n\n\n\n\n\nМагический цвет\n\r\n    На\xa0телевизоре Сяоми А2\xa0можно смотреть все, что угодно\xa0— от\xa0боевиков до\xa0соревнований. Дисплеи 4K\xa0Ultra HD\xa0и\xa0FHD с\xa0охватом цветового пространства DCI-P3 (используется профессионалами\r\n    мировых киностудий), расширенное цветовое пространство и\xa0точная калибровка обеспечивают впечатляющий уровень цветопередачи.\r\n\n\n\n\n\n\n\n\n\n\n\nМощный трехмерный звук\n\r\n    У\xa0Xiaomi A2\xa0два динами

  3%|▎         | 1/38 [00:05<03:19,  5.39s/it]

https://xistore.by/catalog/smart_televizory/televizor_evolution_43_webos/
{'name': 'Телевизор Evolution Smart TV 43" WOS43MR1UHD с WebOS', 'description': 'Открывать новые горизонты\n\r\n    Есть много неплохих телевизоров, но\xa0настоящее удовольствие от\xa0контента можно получить, если купить телевизор Evolution Smart TV\xa043″. У\xa0него большая диагональ 43\xa0дюйма, подсветка D-LED, отличное разрешение\r\n    4К\xa0Ultra\xa0HD (3840×2160\xa0пикселей), контрастность 1200:1 и\xa0широкие углы обзора по\xa0вертикали и\xa0горизонтали 170°/170°.\r\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nНаслаждаться живой картинкой\n\r\n    У\xa0этого телевизора удачно работает технология D-LED. Она отвечает за\xa0яркость и\xa0детализацию картинки, а\xa0также позволяет наслаждаться любым типом контента (включая динамичные сцены) из\xa0всех точек помещения.\r\n    Включенный телевизор словно парит в\xa0пространстве без границ за\xa0счет тонкого безрамочного дизайна.\r\n\n\n\n\n\n\n\n\n\n\n\nПривыкнуть к\xa0комфо

  5%|▌         | 2/38 [00:09<02:40,  4.46s/it]

...

https://xistore.by/catalog/smart_televizory/umnyy_televizor_evolution_55_wos55mr1sbuhd/
{'name': 'Умный телевизор Evolution 55" WOS55MR1SBUHD', 'description': 'Новая эра развлечений\n\r\n    Чтобы каждый день наслаждаться мультимедийным контентом в\xa0качестве\xa04K, достаточно купить телевизор Evolution Smart TV. Вас впечатлят большие диагонали 50/55/65\xa0дюймов, а\xa0также разрешение Ultra\xa0HD (4K,\r\n    3840×2160\xa0пикселей), высокие яркость и\xa0контрастность, отличная детализация и\xa0широкие углы обзора благодаря технологии D-LED.\r\n\n\n\n\n\n\n\n\n\n\n\nВстроенный саунд-бар\n\r\n    Телевизор Эволюшн Смарт ТВ\xa0порадует встроенным саунд-баром с\xa0двумя мощными стереодинамиками общей мощностью 20\xa0Вт. Они поддерживают Dolby Audio и\xa0обеспечивают мощный, объемный и\xa0максимально сбалансированный\r\n    звук. Смотрите любимые сериалы или различный цифровой контент по-новому.\r\n\n\n\n\n\n\n\n\n\n\n\nФирменная операционная система\n\r\n    WebOS TV\xa06.0\xa0— понятная 

 95%|█████████▍| 36/38 [01:48<00:06,  3.11s/it]

https://xistore.by/catalog/smart_televizory/televizor_xiaomi_tv_a_pro_43_2025_4k/
{'name': 'Телевизор Xiaomi TV A Pro 43" 2025 4K', 'description': 'Ярко и\xa0четко\n\r\n    Чтобы с\xa0головой погрузиться в\xa0безграничный мир контента, стоит купить телевизор Xiaomi TV\xa0A\xa0Pro 2025 4K. Гаджет получил яркий 4К\xa0QLED-дисплей с\xa0передовой технологией отображения с\xa0квантовыми точками.\r\n    Благодаря чему телевизор обладает широкой цветовой гаммой и\xa0позволяет наслаждаться потрясающим качеством изображения не\xa0выходя из\xa0дома.\r\n\n\n\n\n\n\n\n\n\n\n\nНикаких границ\n\r\n    Конструкция с\xa0ультра тонкой рамкой мгновенно погружает в\xa0контент, а\xa0металлический корпус станет украшением любого интерьера. На\xa0выбор доступно сразу нелько вариантов диагоналей: 43, 55, 65\xa0и\xa075\xa0дюймов.\r\n\n\n\n\n\n\n\n\n\n\n\nВидеть больше\n\r\n    Создайте кинотеатр не\xa0выходя из\xa0дома с\xa0помощью телевизора Сяоми ТВ\xa0А\xa0Про. Технология MEMC оценивает изображение и\xa0де

 97%|█████████▋| 37/38 [01:51<00:03,  3.13s/it]

https://xistore.by/catalog/smart_televizory/televizor_xiaomi_tv_max_100_4k/
{'name': 'Телевизор Xiaomi TV Max 100 4K', 'description': 'Диагональ экрана 100', 'reviews': [], 'shops_availability': [], 'price': 10499.0, 'full_price': 11930.68, 'brand': 'Главная', 'remind_status': False, 'url': 'https://xistore.by/catalog/smart_televizory/?PAGEN_1=5', 'category': 'Unknown', 'images': ['/upload/resize/element/159496/fd5/b08291c11c7664e6f67d76cd29eebbd0_482_482_80.jpg'], 'characteristics': [{'name': 'Дата выхода на рынок', 'value': '2024 г.'}, {'name': 'Тип', 'value': 'LED'}, {'name': 'Диагональ экрана', 'value': '100"'}, {'name': 'Разрешение', 'value': '3840x2160 (4K UHD)'}, {'name': 'Частота матрицы', 'value': '120 Гц'}, {'name': 'Изогнутый экран', 'value': ''}, {'name': 'Поддержка 3D', 'value': ''}, {'name': 'Smart TV', 'value': ''}, {'name': 'Платформа Smart TV', 'value': 'Android TV'}, {'name': 'Версия системы', 'value': 'Android TV 11'}, {'name': 'Безрамочный дизайн', 'value': ''}, {'n

In [7]:
save_to_csv(all_product_data, filename='televizory.csv')

In [63]:
product_data['category']

'Unknown'

In [42]:
script.find_all('div', class_='pa-result_item')

[<div class="pa-result_item">
 <div class="pa-result_title">ТЦ «Секрет»</div>
 <div class="pa-result_info">
 <div class="pa-result_available">
 <div class="pa-result_info-title">Наличие                                                        :
                                                     </div>
 <div class="count-code-gomel_sekret pa-result_available-res">
 <span></span>
 <span></span>
 <span></span>
 <span></span>
 <span></span>
 </div>
 <div class="promo-apply">
 <span>Промокод применен</span>
 </div>
 </div>
 <div class="pa-result_reserve--big pa-result_reserve fotr-reserv-gomel_sekret">
 <form action="/" method="" style="display:flex;">
 <div class="block_input">
 <input name="shop" type="hidden" value="gomel_sekret"/>
 <input class="js-reservedCheckPersonal" name="check_personal" type="hidden" value="">
 <input id="reservePhone" name="user_phone" placeholder="Ваш телефон" type="tel" value=""/>
 <label class="input_caption" for="reservePhone">Ваш телефон</label>
 </input></d

In [31]:
with open('test.html', 'w') as file:
    file.write(soup.prettify())